In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")


In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")
display(df_trips)
display(df_payments)



In [0]:
silver = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.silver_transform"

df_trips = spark.read.table(f"{silver}.trips")
df_payments = spark.read.table(f"{silver}.payments")
display(df_trips)
display(df_payments)


In [0]:
from pyspark.sql.functions import to_date
df_joined = (
    df_trips
    .join(df_payments, "trip_id", "inner")
    .drop(df_trips.customer_id)     # remove the ambiguous column
)



In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

df_daily_revenue = (
    df_joined
    .groupBy("trip_id")
    .agg(
        sum("amount").alias("total_revenue"),
        countDistinct("trip_id").alias("total_trips"),
        countDistinct("customer_id").alias("unique_customers"),
        avg("amount").alias("avg_revenue_per_trip")
    )
)
display(df_daily_revenue)


In [0]:
from pyspark.sql.functions import to_date

df_joined = df_joined.withColumn("trip_date", to_date("trip_start_time"))


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

df_daily_revenue = (
    df_joined
    .groupBy("trip_date")
    .agg(
        sum("amount").alias("total_revenue"),
        countDistinct("trip_id").alias("total_trips"),
        countDistinct("customer_id").alias("unique_customers"),
        avg("amount").alias("avg_revenue_per_trip")
    )
)
display(df_daily_revenue)


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS 
thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.gold_transform
""")


In [0]:
gold = "thiruvengadam_k_databricks_npmentorskool_onmicrosoft_com.gold_transform"

df_daily_revenue.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{gold}.daily_revenue_summary")
